<a href="https://colab.research.google.com/github/cedric-bit/Geospatial-Data-Science-2023/blob/main/loadDATA509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmium




In [ ]:
!pip install gtfs-realtime-bindings


In [ ]:
!pip install osmnx







In [5]:
!pip install --upgrade protobuf



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 2.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
tensorflow-metadata 1.13.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.1 which is incompatible.


In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
from datetime import datetime
import osmnx as ox
from google.transit import gtfs_realtime_pb2
from google.protobuf import text_format
import pandas as pd
import os
import zipfile
import osmium
import datetime
import matplotlib.pyplot as plt
import folium


# Chemin d'accès au dossier contenant les fichiers GTFS-RT et GTFS
shortcut_path = '/content/drive/MyDrive/Geospatial Data Science 2023'

def read_gtfs_static_file(file_path):
    print(f"Ouverture du fichier {file_path}")
    with zipfile.ZipFile(file_path, 'r') as z:
        stops_df = pd.read_csv(z.open('stops.txt'))
        stop_times_df = pd.read_csv(z.open('stop_times.txt'))
        trips_df = pd.read_csv(z.open('trips.txt'))
        routes_df = pd.read_csv(z.open('routes.txt'))
    
    return stops_df, stop_times_df, trips_df, routes_df

def extract_trip_updates(feed_realtime):
    trip_updates = []

    for entity in feed_realtime.entity:
        if entity.HasField('trip_update'):
            trip_update = entity.trip_update
            trip_id = trip_update.trip.trip_id
            route_id = trip_update.trip.route_id
            for stop_time_update in trip_update.stop_time_update:
                stop_id = stop_time_update.stop_id
                arrival_time = stop_time_update.arrival.time
                departure_time = stop_time_update.departure.time
                delay = stop_time_update.arrival.delay

                trip_updates.append({
                    'trip_id': trip_id,
                    'route_id': route_id,
                    'stop_id': stop_id,
                    'arrival_time': arrival_time,
                    'departure_time': departure_time,
                    'delay': delay
                })

    return trip_updates    

def process_gtfsrt_file(file_path):
    feed = gtfs_realtime_pb2.FeedMessage()
    
    try:
        with open(file_path, 'rb') as f:
            content = f.read()
            feed.ParseFromString(content)
    except Exception as e:
        #print(f"Erreur lors du traitement du fichier {file_path}: {e}")
        return None

    return feed

# Liste de tous les fichiers GTFS et GTFS-realtime
gtfs_files = []
gtfsrt_files = []

# Parcourez tous les sous-dossiers
for subdir, dirs, files in os.walk(shortcut_path):
    for file in files:
        # Vérifiez si le fichier est un fichier GTFS zip
        if file.endswith('.zip'):
            gtfs_file_path = os.path.join(subdir, file)
            gtfs_files.append(gtfs_file_path)
        # Vérifiez si le fichier est un fichier GTFS-realtime
        elif file.endswith('.gtfsrt'):
            gtfsrt_file_path = os.path.join(subdir, file)
            gtfsrt_files.append(gtfsrt_file_path)

# Utilisez les fonctions pour extraire les informations
for gtfs_file in gtfs_files:
    stops_df, stop_times_df, trips_df, routes_df = read_gtfs_static_file(gtfs_file)
    # Utilisez les DataFrames pour effectuer vos analyses et traitements

for gtfsrt_file in gtfsrt_files:
    feed_realtime = process_gtfsrt_file(gtfsrt_file)
    if feed_realtime is not None:
        trip_updates = extract_trip_updates(feed_realtime)
        # Utilisez la liste trip_updates pour effectuer vos analyses et traitements


# Pour les données GTFS statiques


# Pour les données GTFS en temps réel
if 'trip_updates' in locals():
    print("\nTrip Updates:")
    for i, trip_update in enumerate(trip_updates[:5]):
        print(f"Update {i + 1}:")
        print(f"  Trip ID: {trip_update['trip_id']}")
        print(f"  Route ID: {trip_update['route_id']}")
        print(f"  Stop ID: {trip_update['stop_id']}")
        print(f"  Arrival Time: {trip_update['arrival_time']}")
        print(f"  Departure Time: {trip_update['departure_time']}")
        print(f"  Delay: {trip_update['delay']}")
else:
    print("No trip updates available.")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


RuntimeError: ignored

Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-18/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-19/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-20/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-22/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-23/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-29/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-26/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-28/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-27/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 2023/2023-03-30/gtfs.zip
Ouverture du fichier /content/drive/MyDrive/Geospatial Data Science 20